In [1]:
import pandas as pd

#Plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import calendar
import calplot # actually used

# Score model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Model
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor



/Users/jesperhelen/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
url = "https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/main/ProjectAssignmentData/Dataset-PT.csv"
df = pd.read_csv(url,header=1)
df['Date'] = pd.to_datetime(df['Calendar_date'], format='%Y%m%d')


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(10)

s

NameError: name 's' is not defined

In [ ]:
# Use the nunique() function to count the number of unique values in the column
unique_count = df['bus_id'].nunique()

# Print the result
print(f"Number of different numbers in the column: {unique_count}")


In [ ]:
max_delay_row = df[df['arrival_delay'] == df['arrival_delay'].max()]
print(max_delay_row)

In [ ]:
df.info()

In [ ]:
avg_delays = df.groupby('stop_sequence')['arrival_delay'].mean()

# Plotting
plt.figure(figsize=(10,6))
avg_delays.plot(kind='bar', color=plt.cm.Pastel1.colors, zorder=2)  # using Pastel1 colormap for colors

plt.title('Average Arrival Delay by Stop Sequence')
plt.xlabel('Stop Sequence')
plt.ylabel('Average Arrival Delay (minutes)')
plt.xticks(rotation=0)
plt.grid(axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Filter the data
df_filtered = df[df['arrival_delay'].between(-200, 1000)]

plt.figure(figsize=(12, 6))

# Violin plot
sns.violinplot(data=df_filtered, x='time_of_day', y='arrival_delay', hue='day_of_week', order=['OP', 'MP', 'AP'], hue_order=['weekday', 'weekend'], split=True, inner=None, palette="pastel")

# Point plot to indicate means
sns.pointplot(data=df_filtered, x='time_of_day', y='arrival_delay', hue='day_of_week', order=['OP', 'MP', 'AP'], hue_order=['weekday', 'weekend'], dodge=0.532, join=False, palette="dark", markers="D", scale=0.75, ci=None)

plt.title('Arrival Delay Distribution by Time of Day and Day of Week with Mean')
plt.legend(title='Day of Week')
plt.show()


In [ ]:
print(df["weather"].unique())
print(df["temperature"].unique())

df = df.drop(columns='temperature')  # Remove it since we have them already
df = df.drop(columns='weather')  # Remove it since we have them already


In [ ]:
print("routeid",df["route_id"].unique())
print("bus id",df["bus_id"].unique())

print("time of day",df["time_of_day"].unique())
print("day of week",df["day_of_week"].unique())

df = df.drop(columns='time_of_day')  # Remove it since we have them already
df = df.drop(columns='day_of_week')  # Remove it since we have them already



In [ ]:
column_titles_list = df.columns.tolist()
print(column_titles_list)

In [ ]:
# Set a theme for seaborn
sns.set_theme()
filtered_df = df[(df['arrival_delay'] >= -1000) & (df['arrival_delay'] <= 1000)]
# Plot the enhanced histogram
plt.figure(figsize=(5,3))
sns.histplot(filtered_df['arrival_delay'], bins=30, kde=True, color="#ff5722")
plt.title('Arrival Delay Histogram')
plt.xlabel('Arrival Delay')
plt.ylabel('Frequency')
plt.show()


In [ ]:
snow_days_count = df['factor(weather)Snow'].sum()

print(f"Number of snow days: {snow_days_count}")

In [ ]:
df.head()

In [ ]:
# Group by 'Calendar_date' and 'bus_id' to get the average delay
avg_delays = df.groupby(['Date', 'bus_id']).arrival_delay.mean().reset_index()

# Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=avg_delays, x='Date', y='arrival_delay', hue='bus_id', marker="o")
plt.title('Average Arrival Delay by Bus over Time')
plt.ylabel('Average Arrival Delay')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend(title='Bus ID')
plt.tight_layout()
plt.show()

In [ ]:
df['factor(day_of_week)weekend_map'] = df['factor(day_of_week)weekend'].map({0: 'weekday', 1: 'weekend'})

# Group by 'Date' and 'factor(day_of_week)weekend' to get the average delay
avg_delays = df.groupby(['Date', 'factor(day_of_week)weekend_map']).arrival_delay.mean().reset_index()

# Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=avg_delays, x='Date', y='arrival_delay', hue='factor(day_of_week)weekend_map', marker="o")
plt.title('Average Arrival Delay by Day of Week over Time')
plt.ylabel('Average Arrival Delay')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend(title='Day of Week')
plt.tight_layout()
plt.show()

df = df.drop(columns='factor(day_of_week)weekend_map')  # Remove it since we have them already


In [ ]:
# Convert 'Date' column to pandas datetime format
# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Plot using calplot
calplot.calplot(df['factor(temperature)Cold'], cmap='Blues', edgecolor='lightgray', suptitle="Cold Days", linewidth=1)


In [ ]:
# Group by 'Date' and take the mean of the 'arrival_delay'
avg_delay_per_day = df.groupby('Date')['arrival_delay'].mean()

# Plot the average delay in the calendar
calplot.calplot(avg_delay_per_day, cmap='Reds', edgecolor='lightgray', suptitle="Average Daily Delay", linewidth=1)

In [ ]:
#WHAAAAT!!! BAD DATA?!
# Group by 'Date' and take the max of the 'factor(weather)Snow' column
did_it_snow = df.groupby('Date')['factor(weather)Snow'].max()

# Plot the binary calendar heatmap
calplot.calplot(did_it_snow, cmap='Blues', edgecolor='lightgray', suptitle="Snow Days", linewidth=1)

# Create a base line model (Mean)

In [ ]:

# Assuming df is your DataFrame
# 1. Calculate the mean of the actual delays
mean_delay = df['arrival_delay'].mean()

# 2. Create a new column with the mean delay as prediction
df['predicted_delay'] = mean_delay

# 3. Calculate MSE
mse = mean_squared_error(df['arrival_delay'], df['predicted_delay'])
print(f"Mean Squared Error (MSE): {mse}")

# Calculate other metrics if required:
# Mean Absolute Error (MAE)
mae = mean_absolute_error(df['arrival_delay'], df['predicted_delay'])
print(f"Mean Absolute Error (MAE): {mae}")

# R^2 Score (not very meaningful in this context, but can be used)
r2 = r2_score(df['arrival_delay'], df['predicted_delay'])
print(f"R^2 Score: {r2}")
df = df.drop(columns='predicted_delay')
# COMMENT FOR REPORT
# Since r^2 = 1- ssres / sstot
# ssres is actual - pred -> since predis mean denominator and numerator is the same
# sstot is actual - mean


# Feature engineering

In [ ]:
#df['mean_bus_delay'] = df.groupby('bus_id')['arrival_delay'].transform('mean')

# Regression

In [ ]:
# Split the data into training and test sets (30% held out for testing)
X = df.drop('arrival_delay', axis=1)
y = df['arrival_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Define Ridge regression model
ridge = Ridge(alpha=1.0)  # Change alpha as needed

model = LinearRegression()

# Recursive feature elimination with cross-validation
selector = RFECV(estimator=ridge, step=1, cv=5)
selector = selector.fit(X_train, y_train)


# Select the important features based on RFECV
X_train_selected = selector.transform(X_train)

print("Number of best features: ", selector.n_features_)
print("Best features: ", X_train.columns[selector.support_])


In [ ]:

ridge.fit(X_train, y_train)
predictions = ridge.predict(X_test)

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Calculate adjusted R^2
n = len(y_test)
k = X_test.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
mae = mean_absolute_error(y_test,predictions)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2 ): {r2}')
print(f'Adjusted R-squared: {adjusted_r2}')


# DecisionTreeRegresson

In [ ]:
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2}")


# Split weekend & weekday

In [ ]:
# Splitting the DataFrame into weekend and weekday DataFrames

# For weekend
df_weekend = df[df['factor(day_of_week)weekend'] == 1].copy()
df_weekend.drop(columns=['factor(day_of_week)weekend', 'factor(day_of_week)weekday'], inplace=True)

# For weekday
df_weekday = df[df['factor(day_of_week)weekday'] == 1].copy()
df_weekday.drop(columns=['factor(day_of_week)weekend', 'factor(day_of_week)weekday'], inplace=True)


In [ ]:
print(df_weekend.columns)

In [ ]:
X = df_weekday.drop('arrival_delay', axis=1)
y = df_weekday['arrival_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [ ]:
models = [
    ("Decision Tree", DecisionTreeRegressor(), {'max_depth': [3, 5, 7, 9], 'min_samples_split': [2, 5, 10]}),
    #("Random Forest", RandomForestRegressor(), {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7]}),
    #("Gradient Boosting", GradientBoostingRegressor(n_iter_no_change=10, validation_fraction=0.1),
     #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]}),
    ("Linear Regression", LinearRegression(), {})
    #("XGBoost", XGBRegressor(), {'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7]})
]
# Define a DataFrame to hold results
results_df = pd.DataFrame(columns=['Model', 'MSE', 'R2 Score', 'Cross-validated MSE', 'Best Params'])

# A set of pastel colors
colors = ['#A8E6CF', '#DCEDC1', '#FFD3B6', '#FFAAA5', '#FF8B94']

plt.figure(figsize=(12, 8))

# Loop through the models
for idx, (model_name, model, params) in enumerate(models):
    print(f"Evaluating {model_name}...")

    grid_search = GridSearchCV(estimator=model, param_grid=params,
                               scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predictions
    y_pred = best_model.predict(X_test)

    # Evaluation
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validation
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')

    # Append results to the results DataFrame
    results_df.loc[idx] = [model_name, mse, r2, -cv_scores.mean(), grid_search.best_params_]
    
    # Plot
    plt.scatter(y_test, y_pred, color=colors[idx], alpha=0.5, label=model_name)

    print("-------------------------------------------------------")

plt.xlabel('Actual Values')
plt.ylabel('Predictions')
plt.title('Model Predictions vs Actual Values')
plt.legend()
plt.show()

print(results_df)


In [ ]:
X = df_weekday.drop('arrival_delay', axis=1)
y = df_weekday['arrival_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
for col in X.columns:
    print(col)

In [ ]:


import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
filepath=""
%matplotlib inline

# network construction
def construct_network_model():
  model_weekday_NN = Sequential()
  model_weekday_NN.add(Dense(32, activation='linear', input_dim=24))
  model_weekday_NN.add(Dropout(rate=0.01))
  model_weekday_NN.add(Dense(64, activation='linear'))
  model_weekday_NN.add(Dense(1))
  return model_weekday_NN

model_weekday_NN = construct_network_model()
model_weekday_NN.compile(optimizer='adam', loss='mae', metrics=['mae']) # default 'optimizer'
model_weekday_NN.summary()

In [ ]:
# model training -- choose to use callback function

early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=3)
filepath = "weights.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True, mode='min')
hist = model_weekday_NN.fit(X_train, y_train, validation_split=0.2, epochs=400, batch_size=64, callbacks=[early_stop, reduce_lr,checkpoint],verbose=0)

# model evaluation and plot
sns.set()
err = hist.history['mae']
val_err = hist.history['val_mae']
epochs = range(1, len(err) + 1)

plt.plot(epochs, err, '-', label='Training MAE')
plt.plot(epochs, val_err, ':', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend(loc='upper right')
plt.plot()

y_pred = model_weekday_NN.predict(X_test)

mae_M1_NN = mean_absolute_error(y_test, y_pred)
mse_M1_NN = mean_squared_error(y_test, y_pred)
r2_M1_NN = r2_score(y_test, y_pred)

print('This is result of the trained model: ')
# Print the evaluation metrics.
print(f"Mean Absolute Error: {mae_M1_NN}")
print(f"Mean Squared Error: {mse_M1_NN}")
print(f"R-squared: {r2_M1_NN}")

# Load model and evaluation
if not filepath =="":
  model_1 = load_model(filepath)
  y_pred = model_weekday_NN.predict(X_test)
  mae = mean_absolute_error(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print('This is result of the model loaded from the local path: ')

  # Print the calculated metrics.
  print(f"Mean Absolute Error: {mae_M1_NN}")
  print(f"Mean Squared Error: {mse_M1_NN}")
  print(f"R-squared: {r2_M1_NN}")

In [ ]:
  print(f"Mean Absolute Error: {mae_M1_NN}")
  print(f"Mean Squared Error: {mse_M1_NN}")
  print(f"R-squared: {r2_M1_NN}")

In [ ]:
X = df_weekend.drop('arrival_delay', axis=1)
y = df_weekend['arrival_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
models = [
    ("Decision Tree", DecisionTreeRegressor(), {'max_depth': [3, 5, 7, 9], 'min_samples_split': [2, 5, 10]}),
    #("Random Forest", RandomForestRegressor(), {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7]}),
    #("Gradient Boosting", GradientBoostingRegressor(n_iter_no_change=10, validation_fraction=0.1),
     #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]}),
    ("Linear Regression", LinearRegression(), {}),
    #("XGBoost", XGBRegressor(), {'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7]})
]
# Define a DataFrame to hold results
results_df = pd.DataFrame(columns=['Model', 'MSE', 'R2 Score', 'Cross-validated MSE', 'Best Params'])

# A set of pastel colors
colors = ['#A8E6CF', '#DCEDC1', '#FFD3B6', '#FFAAA5', '#FF8B94']

plt.figure(figsize=(12, 8))

# Loop through the models
for idx, (model_name, model, params) in enumerate(models):
    print(f"Evaluating {model_name}...")

    grid_search = GridSearchCV(estimator=model, param_grid=params,
                               scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predictions
    y_pred = best_model.predict(X_test)

    # Evaluation
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validation
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')

    # Append results to the results DataFrame
    results_df.loc[idx] = [model_name, mse, r2, -cv_scores.mean(), grid_search.best_params_]
    
    # Plot
    plt.scatter(y_test, y_pred, color=colors[idx], alpha=0.5, label=model_name)

    print("-------------------------------------------------------")

plt.xlabel('Actual Values')
plt.ylabel('Predictions')
plt.title('Model Predictions vs Actual Values')
plt.legend()
plt.show()

print(results_df)
